<a href="https://colab.research.google.com/github/agatagruza/private-ai/blob/master/SPAIC_Project14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 14: Encrypted Deep Learning

##Reviewing Additive Secret Sharing
For more info about SMPC protocols like this one, visit https://mortendahl.github.io (Below example was pulled with Morten's permission from above website).
**Encode/Decode** - takes a decimal number and encode them using fixed precision. 
**Encrypt** - takes a decrype number and converst it into three hares specifically. 
**Decrypt** does the opposite to encrypt.

In [0]:
pip install syft

In [2]:
import torch as th
import numpy as np
import random
import syft as sy

W0729 00:51:21.629671 139883854743424 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0729 00:51:21.646487 139883854743424 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
BASE = 10
INT_PRECISION = 8
FRACTIONAL_PRECISION = 8
Q = 293973345475167247070445277780365744413

In [0]:
PRECISION = INT_PRECISION + FRACTIONAL_PRECISION
assert(Q > BASE**PRECISION)

In [0]:
def encode(rational):
    upscaled = int(rational * BASE**FRACTIONAL_PRECISION)
    field_element = upscaled % Q
    return field_element

In [0]:
def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**FRACTIONAL_PRECISION
    return rational

In [0]:
def encrypt(secret):
  first = random.randrange(Q)
  second = random.randrange(Q)
  third = (secret - first - second) % Q
  return [first, second, third]

In [0]:
def decrypt(sharing):
  return sum(sharing) % Q

In [0]:
def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [21]:
x = encrypt(encode(6.5))
x

[139302545989461104869649037927381149848,
 168956177393360569247802030954103594063,
 279687967567512820023439486679896744915]

In [22]:
y = encrypt(encode(6.5))
y

[101257153225568791149931919691097707704,
 183295898122160688367087536885261547207,
 9420294127437767553425821204656489502]

In [23]:
z = add(x, y)
z

(240559699215029896019580957618478857552,
 58278730040354010544444290058999396857,
 289108261694950587576865307884553234417)

In [24]:
decrypt(z)

1300000000

In [25]:
decode(decrypt(z))

13.0

In [27]:
25 % BASE

5

##Encrypted Subtraction and Public/Scalar Multiplication

In [0]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [0]:
field = 23740629843760239486723

In [0]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [34]:
(bob_x_share + alices_x_share) % field

5

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [37]:
bob_x_share + alice_x_share

15

In [38]:
bob_y_share + alice_y_share

11

In [39]:
15 + 11

26

In [40]:
15 - 11

4

In [41]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [42]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [43]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [0]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [45]:
(bob_z_share + alice_z_share) % field

4

In [0]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 6

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [46]:
z = bob_x_share + alice_x_share + bob_y_share + alice_y_share
z

26

In [47]:
z % BASE

6

In [0]:
y = 6

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [49]:
z = bob_x_share + alice_x_share + bob_y_share + alice_y_share
z

31

In [50]:
z % BASE

1

In [0]:
x = encrypt(25)
y = encrypt(11)

In [53]:
decrypt(sub(x, y))

14

In [54]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

8

In [0]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [56]:
x = encrypt(encode(5.5))
x

[46949748933161072485910754975423127558,
 242120101164703670671564514268939352770,
 4903495377302503912970008536553264085]

In [0]:
z = imul(x, 3)

In [58]:
decode(decrypt(z))

16.5